In [1]:
# Assuming douban2fb.txt is a mapping file with movie ID to entity ID
mapping_file = 'douban2fb.txt'

# Create a set to store the entity IDs from douban2fb.txt for faster lookup
entity_ids_list = []
movie_id2fb = {}

with open(mapping_file, 'r') as mapping:
    for line in mapping:
        # Assuming the file format is "movieID\tentityID"
        movie_id, entity_id = line.strip().split()
        entity_id="<http://rdf.freebase.com/ns/" + entity_id+ ">"
        movie_id2fb[movie_id] = entity_id
        entity_ids_list.append(entity_id)

In [6]:
import gzip
from tqdm import tqdm
first_jump_entities = []#允许重复，方便后面统计与计算
str_flag="<http://rdf.freebase.com/ns/"
str_fault="<http://rdf.freebase.com/ns/people.person.date_of_birth>"
with gzip.open('freebase_douban.gz', 'rb') as f, gzip.open('output_first_entity.gz', 'wt') as output1, gzip.open('output_first_3.gz', 'wt') as output2:
    for line in tqdm(f, desc="Processing lines", unit="line"):
        triplet = line.strip().decode().split('\t')[:3]
        if triplet[2] in entity_ids_list and str_flag in triplet[0] and str_fault not in triplet[1]:
            first_jump_entities.append(triplet[0])
            output2.write(triplet[0]+" "+triplet[1]+" "+triplet[2]+"\n")
        elif triplet[0] in entity_ids_list and str_flag in triplet[2] and str_fault not in triplet[1]:
            first_jump_entities.append(triplet[2])
            output2.write(triplet[0]+" "+triplet[1]+" "+triplet[2]+"\n")
    for i in first_jump_entities:
        output1.write(i+"\n")


Processing lines: 395577070line [1:18:48, 83656.79line/s]


In [11]:
import gzip
from tqdm import tqdm
first_3 = []#存储第一跳的三元组
first_jump = []#存储第一跳的实体
#将output_first_entity.gz与output_first_3.gz中的实体和三元组存入字典
with gzip.open('output_first_entity.gz', 'rb') as f:
    for line in tqdm(f, desc="Processing lines", unit="line"):
        entity = line.strip().decode()
        first_jump.append(entity)
with gzip.open('output_first_3.gz','rb') as f:
    for line in tqdm(f,desc="Processing lines",unit="line"):
        triplet = line.strip().decode().split(" ")[:3]
        first_3.append(triplet)

print(first_3[4])
print(first_jump[4])

Processing lines: 240073line [00:00, 809323.41line/s]
Processing lines: 240073line [00:00, 435790.07line/s]

['<http://rdf.freebase.com/ns/m.010ntlwj>', '<http://rdf.freebase.com/ns/film.film_crew_gig.film>', '<http://rdf.freebase.com/ns/m.08s6mr>']
<http://rdf.freebase.com/ns/m.010ntlwj>


In [35]:
import copy
#关系<=50则删除，实体<=20则删除
min_relation = 50
min_entity = 30

#函数，通过关系小于50更新first_3，通过first_3更新first_jump
def update_first_3_by_relation (first_3,min_relation,first_jump):
    num_relation = {}#存储每个关系的数量
    num_of_entity = len(first_3)#实体的数量
    for i in range(num_of_entity):
        if first_3[i][1] not in num_relation:
            num_relation[first_3[i][1]] = 1
        else:
            num_relation[first_3[i][1]] = num_relation[first_3[i][1]] + 1
    id_to_delete = []#存储需要删除的三元组的id
    #在first_3中删除关系小于50的三元组,并在first_jump中删除对应的实体
    for i in range(num_of_entity):
        if num_relation[first_3[i][1]] <= min_relation:
            id_to_delete.append(i)
    for i in range(len(id_to_delete)):
        del first_3[id_to_delete[i]-i]
        del first_jump[id_to_delete[i]-i]
    
def update_first_3_by_entity(first_3,min_entity,first_jump):
    num_entity = {}#存储每个实体的数量
    num_of_entity = len(first_3)#实体的数量,即三元组的数量
    for i in range(num_of_entity):
        if first_jump[i] not in num_entity:
            num_entity[first_jump[i]] = 1
        else:
            num_entity[first_jump[i]] = num_entity[first_jump[i]] + 1
    id_to_delete = []#存储需要删除的三元组的id
    #在first_3中删除实体小于20且不在entity_id_list的三元组,并在first_jump中删除对应的实体
    for i in range(num_of_entity):
        if num_entity[first_jump[i]] <= min_entity:
            id_to_delete.append(i)
    for i in range(len(id_to_delete)):
        del first_3[id_to_delete[i]-i]
        del first_jump[id_to_delete[i]-i]
            
#处理第一跳，删除关系小于50的三元组，删除实体小于20的三元组,轮流执行直至first_3和first_jump不再变化
def process_first_jump(first_3,first_jump,min_entity,min_realtion):
    first_3_old = copy.deepcopy(first_3)
    first_jump_old = copy.deepcopy(first_jump)
    update_first_3_by_relation(first_3,min_relation,first_jump)
    update_first_3_by_entity(first_3,min_entity,first_jump)
    while first_3_old != first_3 and first_jump_old != first_jump:
        first_3_old = copy.deepcopy(first_3)
        first_jump_old = copy.deepcopy(first_jump)
        update_first_3_by_relation(first_3,min_relation,first_jump)
        update_first_3_by_entity(first_3,min_entity,first_jump)
    
process_first_jump(first_3,first_jump,min_entity,min_relation)

In [36]:
#检测first_3和first_jump是否一一对应
print(len(first_3))
print(len(first_jump))
for i in range(len(first_3)):
    if first_3[i][0] != first_jump[i] and first_3[i][2] != first_jump[i]:
        print(i)
        print(first_3[i][0])
        print(first_jump[i])
        print("error")

38414
38414


In [40]:
#测试正确性
num_entity_first_jump = {}
num_relation_first_jump = {}
num_of_entity = len(first_jump)
for i in range(num_of_entity):
    if first_jump[i] not in num_entity_first_jump:
        num_entity_first_jump[first_jump[i]] = 1
    else:
        num_entity_first_jump[first_jump[i]] = num_entity_first_jump[first_jump[i]] + 1
    if first_3[i][1] not in num_relation_first_jump:
        num_relation_first_jump[first_3[i][1]] = 1
    else:
        num_relation_first_jump[first_3[i][1]] = num_relation_first_jump[first_3[i][1]] + 1
print(len(num_relation_first_jump))
# print(len(num_relation_first_jump))
print(len(num_entity_first_jump))


22
200


In [38]:
#将first_jump与entity_id_list合并
first_jump_final = first_jump + entity_ids_list
#第一跳收尾，将first_jump变成不含重复实体的列表
first_jump_final = list(set(first_jump_final))
print(len(first_jump_final))
#将first_jump写入文件
with gzip.open('first_jump.gz','wb') as f:
    for i in first_jump_final:
        f.write((i+"\n").encode())
#将first_3写入文件
with gzip.open('first_3.gz','wb') as f:
    for i in range(len(first_3)):
        f.write((first_3[i][0]+" "+first_3[i][1]+" "+first_3[i][2]+"\n").encode())

778


In [ ]:
import gzip
from tqdm import tqdm
#将first_jump作为第二跳的起点，保存在second_initial中
with gzip.open('first_jump.gz','rb') as f:
    second_initial = []
    for line in tqdm(f, desc="Processing lines", unit="line"):
        second_initial.append(line.strip().decode())

#开始第二跳
second_jump_entities = []
str_flag="<http://rdf.freebase.com/ns/"
str_fault="<http://rdf.freebase.com/ns/people.person.date_of_birth>"
with gzip.open('freebase_douban.gz','rb') as f, gzip.open('output_second_entity.gz','wt') as output1, gzip.open('output_second_3.gz','wt') as output2:
    for line in tqdm(f, desc="Processing lines", unit="line"):
        triplet = line.strip().decode().split('\t')[:3]
        if triplet[2] in second_initial and str_flag in triplet[0] and str_fault not in triplet[1]:
            second_jump_entities.append(triplet[0])
            output2.write(triplet[0]+" "+triplet[1]+" "+triplet[2]+"\n")
        elif triplet[0] in second_initial and str_flag in triplet[2] and str_fault not in triplet[1]:
            second_jump_entities.append(triplet[2])
            output2.write(triplet[0]+" "+triplet[1]+" "+triplet[2]+"\n")
    for i in second_jump_entities:
        output1.write(i+"\n")


In [ ]:
import gzip
from tqdm import tqdm
second_3 = {}
second_jump = []
second_3_flag = 0
with gzip.open('output_second_entity.gz','rb') as f:
    for line in tqdm(f, desc="Processing lines", unit="line"):
        entity = line.strip().decode()
        second_jump.append(entity)
with gzip.open('output_second_3.gz','rb') as f:
    for line in tqdm(f, desc="Processing lines", unit="line"):
        triplet = line.strip().decode().split(" ")[:3]
        second_3[second_3_flag] = []
        second_3[second_3_flag].append(triplet[0])
        second_3[second_3_flag].append(triplet[1])
        second_3[second_3_flag].append(triplet[2])
        second_3_flag = second_3_flag + 1

In [ ]:
import copy
#先过滤掉出现超过 2w 次的实体和出现少于 50 次的关系
min_relation = 50
max_entity = 20000
num_relation = {}#存储每个关系的数量
num_entity = {}#存储每个实体的数量
for i in second_3:
    if second_3[i][1] not in num_relation:
        num_relation[second_3[i][1]] = 1
    else:
        num_relation[second_3[i][1]] = num_relation[second_3[i][1]] + 1
for i in second_jump:
    if i not in num_entity:
        num_entity[i] = 1
    else:
        num_entity[i] = num_entity[i] + 1

for i in second_3:
    if num_relation[second_3[i][1]] < min_relation:
        del second_3[i]
        del second_jump[i]
    if num_entity[second_3[i][0]] > max_entity:
        del second_3[i]
        del second_jump[i]
    elif num_entity[second_3[i][2]] > max_entity:
        del second_3[i]
        del second_jump[i]

In [ ]:
min_entity = 15
min_relation = 50
#函数，通过关系小于50更新second_3，通过second_3更新second_jump
def update_second_3_by_relation (second_3,min_relation,second_jump):
    num_relation = {}#存储每个关系的数量
    for i in second_3:
        if second_3[i][1] not in num_relation:
            num_relation[second_3[i][1]] = 1
        else:
            num_relation[second_3[i][1]] = num_relation[second_3[i][1]] + 1
    #在second_3中删除关系小于50的三元组,并在second_jump中删除对应的实体
    for i in second_3:
        if num_relation[second_3[i][1]] < min_relation:
            del second_3[i]
            del second_jump[i]

def update_second_3_by_entity(second_3,min_entity,second_jump):
    num_entity = {}#存储每个实体的数量
    for i in second_jump:
        if i not in num_entity:
            num_entity[i] = 1
        else:
            num_entity[i] = num_entity[i] + 1
    #在second_3中删除实体小于15且不在second_initial的三元组,并在second_jump中删除对应的实体
    for i in second_3:
        if num_entity[second_3[i][0]] < min_entity and second_3[i][0] not in second_initial:
            del second_3[i]
            del second_jump[i]
        elif num_entity[second_3[i][2]] < min_entity and second_3[i][2] not in second_initial:
            del second_3[i]
            del second_jump[i]

def process_second_jump(second_3,second_jump,min_entity,min_relation):
    second_3_old = copy.deepcopy(second_3)
    second_jump_old = copy.deepcopy(second_jump)
    update_second_3_by_relation(second_3,min_relation,second_jump)
    update_second_3_by_entity(second_3,min_entity,second_jump)
    while second_3_old != second_3 and second_jump_old != second_jump:
        second_3_old = copy.deepcopy(second_3)
        second_jump_old = copy.deepcopy(second_jump)
        update_second_3_by_relation(second_3,min_relation,second_jump)
        update_second_3_by_entity(second_3,min_entity,second_jump)
    return second_3,second_jump

second_3,second_jump = process_second_jump(second_3,second_jump,min_entity,min_relation)


In [ ]:
#处理第二跳的收尾，将second_jump变成不含重复实体的列表
second_jump = list(set(second_jump))
#将second_jump与second_initial合并
second_jump = second_jump + second_initial
print(len(second_jump))
#将second_3写入文件graph_2step.gz 
with gzip.open('graph_2step.gz','wb') as f:
    for i in second_3:
        f.write((second_3[i][0]+" "+second_3[i][1]+" "+second_3[i][2]+"\n").encode())

